<center><h1>Segmenting and Clustering Neighborhoods in Toronto</h1></center>

<h2>Contents</h2>

Hi There. This notebook is divided into 3 sections and are as follows:
<ul>
    <li>Web Scraping Toronto Neighborhood Data</li>
    <li>Grabbing coordinates of Neighborhoods</li>
    <li>Analysis of Neighborhoods</li>
</ul>
        

<h3> Part 1: Web Scraping Toronto Neighborhood Data</h3>

<h5>Required Libraries</h5>
<ul>
    <li>Selenium</li>
    <li>Beautiful Soup</li>
    <li>Pandas</li>
    </u>

In [8]:
# Installing Selenium
import os
os.system('pip3 install selenium')

0

In [10]:
# Installing BeautifulSoup
os.system('pip3 install BeautifulSoup4')

0

In [9]:
# Installing Pandas if not present
os.system('pip3 install pandas')

0

In [1]:
# Importing Installed Libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

In [2]:
# pulling Wikipedia page
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wiki_page

<Response [200]>

In [3]:
# Parsing the page with BeautifulSoup
soup = BeautifulSoup(wiki_page.content, 'html.parser')

In [4]:
# Extracting children of wiki_page
wiki_page_children = list(soup.children)
wiki_table = soup.findAll('table','wikitable')

In [38]:
os.system('pip3 install lxml')

0

<b>Reading Data present in the table into a DataFrame</b>

In [5]:
# Reading Html Table Data into DataFrame
data_frame = pd.read_html(str(wiki_table))[0]

In [6]:
data_frame.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<b>Cleaning the DataFrame based on below rules:</b>
<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    <li>More than one neighborhood can exist in one postal code area.</li>
    <li>These rows will be combined into one row with the neighborhoods separated with a comma.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
</ul>

In [7]:
# Get names of indexes for which column Borourgh is Not Assigned
indexNames = data_frame[ data_frame['Borough'] == 'Not assigned' ].index
 
# Delete these row indexes from dataFrame
data_frame.drop(indexNames , inplace=True)
data_frame.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
# Grouping by postal code
grouped_data = data_frame.groupby(['Postal code'])
grouped_data = grouped_data.apply(pd.DataFrame)
grouped_data['Neighborhood'] = grouped_data['Neighborhood'].str.replace('/',',')
grouped_data['Neighborhood'] = grouped_data['Neighborhood'].str.replace(" , ", ",")
grouped_data.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [9]:
# Reseting Index
grouped_data = grouped_data.reset_index()
grouped_data.drop('index', axis=1, inplace=True)
grouped_data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [16]:
grouped_data.shape

(103, 3)